# Imports

In [7]:
%matplotlib inline
import io
import obspy
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import requests
from soundfile import SoundFile
from modest_image import imshow

matplotlib.rcParams['figure.figsize'] = (25.0, 30.0)

Get the data

In [8]:
url = 'https://rawdata.oceanobservatories.org/files/CE02SHBP/LJ01D/11-HYDBBA106/2015/12/16/OO-HYEA2--YDH-2015-12-16T02:20:00.000000.mseed'
# stream = obspy.read(url, ssl_verify=False)  # Fetch from data server
stream = obspy.read(url)  # Read local file

In [ ]:
def stft(data, fs, fft_size=1024, overlap_fac=.5):
    # from https://kevinsprojects.wordpress.com/2014/12/13/short-time-fourier-transform-using-python-and-numpy/
    # data = a numpy array containing the signal to be processed
    # fs = a scalar which is the sampling frequency of the data

    hop_size = np.int32(np.floor(fft_size * (1-overlap_fac)))
    pad_end_size = fft_size          # the last segment can overlap the data by no more than one window size
    total_segments = np.int32(np.ceil(len(data) / np.float32(hop_size)))
    t_max = len(data) / np.float32(fs)

    window = np.hanning(fft_size)  # our half cosine window
    inner_pad = np.zeros(fft_size) # the zeros which will be used to double each segment size

    proc = np.concatenate((data, np.zeros(pad_end_size)))              # the data to process
    result = np.empty((total_segments, fft_size), dtype=np.float32)    # space to hold the result

    for i in xrange(total_segments):                      # for each segment
        current_hop = hop_size * i                        # figure out the current segment offset
        segment = proc[current_hop:current_hop+fft_size]  # get the current segment
        windowed = segment * window                       # multiply by the half cosine function
        padded = np.append(windowed, inner_pad)           # add 0s to double the length of the data
        spectrum = np.fft.fft(padded) / fft_size          # take the Fourier Transform and scale # samples
        autopower = np.abs(spectrum * np.conj(spectrum))  # find the autopower spectrum
        result[i, :] = autopower[:fft_size]               # append to the results array

    result = 20*np.log10(result)          # scale to db
    result = np.clip(result, -40, 200)    # clip values
    return result

def specgram(data):
    X = stft(data, 64000)
    vmin = np.nanpercentile(X, 5)
    vmax = np.nanpercentile(X, 95)
    num_plots = 10
    fig, axes = plt.subplots(num_plots, sharex=False, sharey=True)
    
    x, y = X.shape
    
    yticks = np.linspace(0, y, 9)
    yticklabels = np.round(np.linspace(0, 32000, 9)).astype(int)
    
    step = len(X)/num_plots
    for index in range(num_plots):
        start = index * step
        stop = start + step
        data_slice = X[start:stop]
        ax = axes[index]
        ax.set_ylabel('freq (Hz)')
        ax.set_yticks(yticks)
        ax.set_yticklabels(yticklabels)
        data_axes = plt.imshow(ax, data_slice.T, origin='lower', cmap='jet', interpolation='nearest',
                     aspect='auto', vmin=vmin, vmax=vmax)

    cax = fig.add_axes([0.965, 0.12, 0.01, 0.775])
    cb = fig.colorbar(data_axes, cax=cax, use_gridspec=True)
    cb.set_label('dB')

Create a spectrogram

In [ ]:
specgram(stream[0].data)

Filter out the DC component of the signal

In [ ]:
filtered = stream.copy()
filtered.filter('highpass', freq=1000.0)

In [ ]:
stream.plot()

In [ ]:
filtered.plot()

Write as WAV

In [ ]:
filtered.write('voices.wav', format='WAV', framerate=64000, width=2)

In [ ]:
stream.write('voices32.wav', format='WAV', framerate=64000)

Write as FLAC

In [ ]:
with SoundFile('test24.flac', 'w', 64000, 1, 'PCM_24') as f:
     f.write(stream[0].data / ((2.0**23)-1))

Write an amplified FLAC 

In [ ]:
filtered.normalize()
with SoundFile('amped.flac', 'w', 64000, 1) as f:
    f.write(filtered[0].data)